In [59]:
import pandas as pd
import sklearn

In [60]:
sentences = pd.read_csv("/data/semeval-2020/task-11/processed/cleaned_sentences.csv")

In [61]:
sentences.head()

,cleaned_sentence,propaganda,classes
0,Iran Admits To Aiding Al-Qaeda and Facilitatin...,False,[]
1,But now we have definitive confirmation.,False,[]
2,It was Iran Bush should have invaded after 9/1...,False,[]
3,"Now consider this: even though, as President o...",False,[]
4,The Iran nuclear deal should never have procee...,True,[6]


In [63]:
sentences.columns = ["text","label","classes"]
sentences[["label","text"]].to_csv("/data/semeval-2020/task-11/processed/cleaned_sentences_altered.csv",index=False)

In [64]:
from sklearn.model_selection import StratifiedShuffleSplit

In [65]:
split = list(StratifiedShuffleSplit(n_splits=1,test_size=0.2).split(sentences,sentences.label))

In [66]:
for train_ix,test_ix in split:
    test = sentences.iloc[test_ix]
    train = sentences.iloc[train_ix]

In [67]:
# test

In [68]:
# sentences.propaganda.value_counts()

In [69]:
from fastai.text import *

In [70]:
# data_lm = TextLMDataBunch.from_df("/data/semeval-2020/task-11/processed/",train_df=train,valid_df=test,text_cols=0)\
#             .label_for_lm()\
#             .databunch(bs=128)\


In [71]:
data_lm = TextLMDataBunch.from_csv("/data/semeval-2020/task-11/processed/","cleaned_sentences_altered.csv")

In [72]:
data_lm.show_batch()

idx,text
0,"party lines to advance the nomination of xxmaj judge xxmaj brett xxmaj kavanaugh out of committee to the xxmaj senate floor for a vote . xxbos xxmaj the assignment also included a pro - xxmaj palestinian , anti - xxmaj israel article . xxbos xxmaj if xxmaj trump and his generals felt our vital interests could not allow xxmaj syria and xxmaj iraq to drift into the orbit of xxmaj"
1,"line . xxbos xxmaj the story of the xxmaj oakland mayor who tipped off illegal alien criminals to xxup ice raids resulting in rapists and child sex offenders escaping arrest just keeps getting worse . xxbos xxmaj this timeless truth of human nature is one that we xxunk ca n’t seem to accept . xxbos xxmaj and because he saw it pleased the xxmaj jews , he proceeded further to"
2,"the xxmaj jewish question . ” xxbos xxmaj it is said that they annihilated 50 - 100 hundred million , but nobody ever mentions this . [ … ] “ xxbos xxmaj even though not everything that ’s gone wrong is xxmaj trump ’s fault , he has managed to make a complete xxunk of xxmaj brett xxmaj kavanaugh ’s nomination to fill the empty seat on the court ."
3,"and to xxmaj god xxmaj almighty , not only to protect children and the youth from sexual predators in the xxmaj church , but our seminarians , university students , and all the faithful as well . xxbos for that indeed a notable xxunk hath been done by them is manifest to all them that xxunk in xxmaj jerusalem ; and we can not deny it . xxbos xxmaj you"
4,"on xxmaj friday provides just a fraction of the $ xxunk billion xxmaj rossello has said the island needs to recover from cataclysmic damage to its infrastructure and housing stock , although xxmaj puerto xxmaj rico is xxunk to participate in other programs that could increase the aid to $ 45 billion , according to xxmaj friday 's statement . xxbos xxmaj in xxmaj july 1999 , the monks of"


In [73]:
learn = language_model_learner(data_lm,AWD_LSTM,drop_mult=0.3)

In [74]:
#learn.lr_find()
#learn.recorder.plot_lr(suggestion=True)

In [75]:
learn.fit_one_cycle(5,1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.835905,4.545141,0.260806,00:11
1,4.456148,4.137866,0.289834,00:12
2,4.140226,3.940868,0.300538,00:12
3,3.951661,3.866513,0.306991,00:12
4,3.883932,3.854526,0.308360,00:12


In [76]:
learn.save('first_fit')

In [77]:
learn.unfreeze()

In [78]:
learn.fit_one_cycle(5,1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.680057,3.614512,0.351573,00:15
1,3.457955,3.487682,0.363444,00:15
2,3.142824,3.439617,0.370302,00:15
3,2.831341,3.449083,0.372991,00:15
4,2.624883,3.470540,0.372068,00:15


In [79]:
learn.save('fine_tuned',return_path=True)

PosixPath('/data/semeval-2020/task-11/processed/models/fine_tuned.pth')

In [90]:
learn.load('/data/semeval-2020/task-11/processed/models/fine_tuned')

RuntimeError: Error(s) in loading state_dict for SequentialRNN:
	Missing key(s) in state_dict: "0.module.encoder.weight", "0.module.encoder_dp.emb.weight", "0.module.rnns.0.weight_hh_l0_raw", "0.module.rnns.0.module.weight_ih_l0", "0.module.rnns.0.module.weight_hh_l0", "0.module.rnns.0.module.bias_ih_l0", "0.module.rnns.0.module.bias_hh_l0", "0.module.rnns.1.weight_hh_l0_raw", "0.module.rnns.1.module.weight_ih_l0", "0.module.rnns.1.module.weight_hh_l0", "0.module.rnns.1.module.bias_ih_l0", "0.module.rnns.1.module.bias_hh_l0", "0.module.rnns.2.weight_hh_l0_raw", "0.module.rnns.2.module.weight_ih_l0", "0.module.rnns.2.module.weight_hh_l0", "0.module.rnns.2.module.bias_ih_l0", "0.module.rnns.2.module.bias_hh_l0", "1.layers.0.weight", "1.layers.0.bias", "1.layers.0.running_mean", "1.layers.0.running_var", "1.layers.2.weight", "1.layers.2.bias", "1.layers.4.weight", "1.layers.4.bias", "1.layers.4.running_mean", "1.layers.4.running_var", "1.layers.6.weight", "1.layers.6.bias". 
	Unexpected k

In [32]:
def labelling_function(a):
    print(a)
    return 1

In [82]:
data_clas = TextClasDataBunch.from_df("/data/semeval-2020/task-11/processed/",
                                      vocab=data_lm.vocab,
                                      train_df=train, valid_df=test,
                                     label_cols=1,
                                      text_cols=["text"],bs=128)    

In [91]:
# data_clas = TextClasDataBunch.from_csv("/data/semeval-2020/task-11/processed/","cleaned_sentences_altered.csv", vocab=data_lm.train_ds.vocab, bs=32)\
#             .split_from_df(col=1)\
#             .label_from_df(cols=1,classes=["True","False"])\
#             .databunch()

In [100]:
data = TextClasDataBunch.from_csv("/data/semeval-2020/task-11/processed/", 'cleaned_sentences_altered.csv',vocab=data_lm.vocab)
data.show_batch()

text,target
xxbos xxup breaking xxup news : xxup trump xxup tells xxup jeff xxup sessions xxup to xxup put xxup an xxup end xxup to xxup russia xxup probe xxup as xxup he xxup claims xxup prosecutors xxup doing xxup mueller xxup ’s ‘ xxup dirty xxup work xxup are a xxup xxunk xxup to xxup america xxmaj white xxmaj house xxup xxunk say if xxmaj trump will order xxmaj jeff,True
"xxbos xxmaj supporting xxmaj local xxmaj efforts to xxmaj reduce and xxmaj combat xxmaj gun xxmaj crimes , including : xxmaj opposing concealed carry xxunk policies and legislation that would xxunk city policies established to protect residents ; xxmaj providing local governments and law enforcement officials access to xxup xxunk gun trace data ; xxmaj opposing "" xxmaj stand xxmaj your xxmaj xxunk or "" xxmaj shoot xxmaj xxunk laws",False
"xxbos xxmaj as far as the xxmaj roman xxmaj curia is concerned , for the moment i will stop here , even if the names of other prelates in the xxmaj vatican are well known , even some very close to xxmaj pope xxmaj francis , such as xxmaj cardinal xxmaj francesco xxmaj coccopalmerio and xxmaj archbishop xxmaj vincenzo xxmaj xxunk , who belong to the homosexual current in favor",False
"xxbos xxmaj hungary 's xxmaj orban calls for global anti - migrant alliance with eye on 2018 elections ( xxmaj xxunk xxunk ) * xxup pm xxmaj orban faces elections on xxmaj april 8 , leads all polls * xxmaj migration has been xxmaj orban 's defining theme since 2015 * xxmaj says will counter globalism alongside xxup u.s. , xxup uk , xxmaj israel xxmaj by xxmaj marton xxmaj",False
"xxbos xxup fbi xxmaj informant “ xxmaj threatened ” xxmaj after xxmaj offering xxmaj details xxmaj linking xxmaj clinton xxmaj foundation xxmaj to xxmaj russian xxmaj bribery xxmaj case xxmaj this report was originally published by xxmaj tyler xxmaj durden at xxmaj zero xxmaj hedge xxmaj while the mainstream media has largely ignored it , the scandal surrounding xxmaj russian efforts to acquire 20 % of xxmaj america ’s uranium",True


In [101]:
data_clas.batch_size

128

In [102]:
data_clas.classes

[False, True]

In [103]:
# data_clas.vocab.itos = data_lm.vocab.itos

In [106]:
learn = text_classifier_learner(data, AWD_LSTM,drop_mult=0.3)

In [109]:
torch.load('/data/semeval-2020/task-11/processed/models/fine_tuned.pth')

{'model': OrderedDict([('0.encoder.weight',
               tensor([[-1.7455e-01, -4.9872e-02,  1.0374e-01,  ..., -6.7403e-02,
                         5.1643e-01,  1.3855e-02],
                       [ 1.0644e-02, -1.6383e-02,  2.1671e-02,  ...,  7.7826e-03,
                         7.0390e-02, -3.0846e-02],
                       [ 3.5604e-01, -6.7456e-01, -5.6266e-04,  ..., -5.1386e-02,
                        -1.4043e-01,  1.3804e-01],
                       ...,
                       [ 2.4249e-01,  2.2241e-01,  2.4234e-01,  ..., -2.1878e-01,
                         2.7774e-01,  3.0528e-01],
                       [-1.4467e-02,  6.2617e-02, -1.7430e-02,  ..., -1.8095e-01,
                         1.3605e-01, -1.8037e-01],
                       [ 2.0782e-02,  1.6795e-01, -8.9097e-02,  ...,  5.5658e-02,
                        -1.8048e-01, -2.6070e-02]], device='cuda:0')),
              ('0.encoder_dp.emb.weight',
               tensor([[-1.7455e-01, -4.9872e-02,  1.0374e-01,  ...,

In [105]:
learn.load_encoder('fine_tuned')

RuntimeError: Error(s) in loading state_dict for AWD_LSTM:
	Missing key(s) in state_dict: "encoder.weight", "encoder_dp.emb.weight", "rnns.0.weight_hh_l0_raw", "rnns.0.module.weight_ih_l0", "rnns.0.module.weight_hh_l0", "rnns.0.module.bias_ih_l0", "rnns.0.module.bias_hh_l0", "rnns.1.weight_hh_l0_raw", "rnns.1.module.weight_ih_l0", "rnns.1.module.weight_hh_l0", "rnns.1.module.bias_ih_l0", "rnns.1.module.bias_hh_l0", "rnns.2.weight_hh_l0_raw", "rnns.2.module.weight_ih_l0", "rnns.2.module.weight_hh_l0", "rnns.2.module.bias_ih_l0", "rnns.2.module.bias_hh_l0". 
	Unexpected key(s) in state_dict: "model", "opt". 

In [99]:
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,0.680337,0.643640,0.674590,00:08
1,0.637769,0.626233,0.674269,00:07
2,0.615484,0.625123,0.674269,00:08
3,0.621291,0.632317,0.674269,00:07


KeyboardInterrupt: 

In [21]:
a = pd.read_csv("/data/semeval-2020/task-11/processed/sentence_dataset.csv")

In [24]:
a.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
15560    False
15561    False
15562    False
15563    False
15564    False
Length: 15565, dtype: bool

In [31]:
a[~a.isnull().any(axis=1)].to_csv("/data/semeval-2020/task-11/processed/cleaned_sentences.csv",index=False)

In [29]:
data_clas = TextClasDataBunch.from_/data/semeval-2020/data/semeval-2020/task-11/processed/",train_df=a[~a.isnull().any(axis=1)], vocab=data_lm.train_ds.vocab, bs=32,text_cols="cleaned_sentence",label_cols="propaganda")

TypeError: from_df() missing 1 required positional argument: 'valid_df'